# Fundamentals of Information Systems

## Python Programming (for Data Science)

### Master's Degree in Data Science

#### Giorgio Maria Di Nunzio
#### (Courtesy of Gabriele Tolomei FIS 2018-2019)
<a href="mailto:giorgiomaria.dinunzio@unipd.it">giorgiomaria.dinunzio@unipd.it</a><br/>
University of Padua, Italy<br/>
2021/2022<br/>

# Lecture 13: The Classification Problem - Example (Part 1)

## Instructions

-  We consider the dataset file <code>**dataset.csv**</code>, which is contained in the <code>**loan-prediction**</code> directory on the Moodle page.

-  A description of the dataset is available in the <code>**README.txt**</code> file on the same directory.

-  **GOAL:** Use information from past loan applicants contained in <code>**dataset.csv**</code> to predict whether a _new_ applicant should be granted a loan or not.

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Import stats module from scipy, which contains a large number 
# of probability distributions as well as an exhaustive library of statistical functions.
import scipy.stats as stats

%matplotlib inline

# 1. Data Collection

In [ ]:
# Path to the local dataset file (YOURS MAY BE DIFFERENT!)
DATASET_PATH = "./data/loan-prediction/dataset.csv"

In [ ]:
# Load the dataset with Pandas
data = pd.read_csv(DATASET_PATH, sep=",")
print("Shape of the dataset: {}".format(data.shape))
data.head()
# NOTE: the first line of the file is considered as the header

In [ ]:
# Let's have a look at the output of the `describe()` function.
data.describe()

## Observations from the ouput of <code>describe()</code> (Numerical Variables)

-  <code>**LoanAmount**</code> has 614-592 = **22** missing values
-  <code>**Loan_Amount_Term**</code> has 614-600 = **14** missing values
-  <code>**Credit_History**</code> has 614-564 = **50** missing values
-  We can also look that about 84% applicants have a credit history. How? The mean of <code>**Credit_History**</code> field is 0.84 (Remember, <code>**Credit_History**</code> has value 1 for those who have a credit history and 0 otherwise)
-  The <code>**ApplicantIncome**</code> distribution seems to be in line with expectation. Same with <code>**CoapplicantIncome**</code>.


In [ ]:
# Let's have a look at the output of the `describe(include='all')` function.
data.describe(include='all')

## Observations from the ouput of <code>describe(include='all')</code> (Categorical Variables)

-  <code>**Loan_ID**</code> has 614 **unique** values (will use this as our _index_ column)
-  <code>**Gender**</code> has 614-601 = **13** missing values
-  <code>**Married**</code> has 614-611 = **3** missing values
-  <code>**Dependents**</code> has 614-599 = **15** missing values
-  <code>**Self_Employed**</code> has 614-582 = **32** missing values

In [ ]:
# Load the dataset with Pandas
data = pd.read_csv(DATASET_PATH, sep=",", index_col="Loan_ID")
print("Shape of the dataset: {}".format(data.shape))
data.head()
# NOTE: the first line of the file is considered as the header

## Double-check for any Missing Values

In [ ]:
# Check if there is any missing value in the whole dataset
print("There are missing values in the dataset: {}".
     format(data.isnull().any().any()))

In [ ]:
# Check how missing values are distributed across each variable (i.e., column)
data.apply(lambda x: sum(x.isnull()))

# 2. Data Exploration

## 2.1 Analysis of Data Distributions: Continous Values

-  Let's start visualizing the distributions of the **3 continuous-valued** features:
    - <code>**ApplicantIncome**</code>
    - <code>**CoapplicantIncome**</code>
    - <code>**LoanAmount**</code> (contains 22 NAs)

In [ ]:
# Create a lambda function which will be applied to each entry
# of the numpy 2-D array of AxesSubplot objects
# x is a reference to an AxesSubplot object
y_labeler = lambda x: x.set_ylabel('density')

# np.vectorize() allows calling the function on each element
y_labeler = np.vectorize(y_labeler)

In [ ]:
# Create a Figure containing 1x3 subplots
fig, axes = plt.subplots(1, 3, figsize=(16,6))
# Call the vectorized function for labeling all the y-axes
y_labeler(axes)
# Plot 'ApplicantIncome' on the first subplot
sns.distplot(data.ApplicantIncome, color='#808080', ax=axes[0], 
                 hist_kws=dict(edgecolor="#404040", linewidth=1))

# Plot 'CoapplicantIncome' on the second subplot
sns.distplot(data.CoapplicantIncome, color='#0033cc', ax=axes[1], 
                 hist_kws=dict(edgecolor="k", linewidth=1))

# Plot 'LoanAmount' (limited only to non-NA values) on the third and last subplot
sns.distplot(data.loc[data.LoanAmount.notnull(), 'LoanAmount'], 
                 color='#df2020', ax=axes[2], 
                 hist_kws=dict(edgecolor="#404040", linewidth=1))

# Adjust space between plots
plt.subplots_adjust(wspace=.3, hspace=.3)

## Observations

-  All the three distributions show the presence of some extreme values.

-  To investigate better how those are shaped, let's create some boxplots...

In [ ]:
# Let's produce the boxplots corresponding to the distribution plots above
# Create a Figure containing 1x3 subplots
fig, axes = plt.subplots(1, 3, figsize=(14,8))

sns.boxplot(data.ApplicantIncome, color='#808080', ax=axes[0], orient="v")
sns.boxplot(data.CoapplicantIncome, color='#0033cc', ax=axes[1], orient="v")
sns.boxplot(data.loc[data.LoanAmount.notnull(), 'LoanAmount'], 
                color='#df2020', ax=axes[2], orient="v")
plt.subplots_adjust(wspace=.4, hspace=.3)

In [ ]:
# Let's see if we can spot where those outliers are located
# w.r.t. other features (e.g., Education)
fig, axes = plt.subplots(3, 1, figsize=(8,20))

sns.boxplot(x=data.Education, y=data.ApplicantIncome, palette=sns.color_palette("RdBu", n_colors=2), ax=axes[0])

sns.boxplot(x=data.Education, y=data.CoapplicantIncome, palette=sns.color_palette("RdBu", n_colors=2), ax=axes[1])

sns.boxplot(x=data.Education, y=data.loc[data.LoanAmount.notnull(), 'LoanAmount'], 
                palette=sns.color_palette("RdBu", n_colors=2), ax=axes[2])

plt.subplots_adjust(wspace=.3, hspace=.3)

In [ ]:
# Let's see if we can spot where those outliers are located
# w.r.t. other features (e.g., Gender)
fig, axes = plt.subplots(3, 1, figsize=(8,20))

sns.boxplot(x=data.Gender, y=data.ApplicantIncome, palette=sns.color_palette("hls", n_colors=2), ax=axes[0])

sns.boxplot(x=data.Gender, y=data.CoapplicantIncome, palette=sns.color_palette("hls", n_colors=2), ax=axes[1])

sns.boxplot(x=data.Gender, y=data.loc[data.LoanAmount.notnull(), 'LoanAmount'], 
                palette=sns.color_palette("hls", n_colors=2), ax=axes[2])

plt.subplots_adjust(wspace=.3, hspace=.3)

In [ ]:
# The same plots can be produced using factorplot. Let's see only the last example (Gender)

sns.catplot(kind='box',          # Boxplot
                       x='Gender',          # X-axis - first factor
                       y='ApplicantIncome', # Y-axis - values for boxplot
                       #hue='Education',    # Second factor denoted by color
                       data=data,           # Dataframe 
                       height=8,              # Figure size (x100px)      
                       aspect=1.5,          # Width = size * aspect 
                       legend_out=False,    # Make legend inside the plot
                       palette=sns.color_palette("hls", n_colors=2)
                      )

sns.catplot(kind='box',            # Boxplot
                   x='Gender',            # X-axis - first factor
                   y='CoapplicantIncome', # Y-axis - values for boxplot
                   #hue='Education',      # Second factor denoted by color
                   data=data,             # Dataframe 
                   height=8,                # Figure size (x100px)      
                   aspect=1.5,            # Width = size * aspect 
                   legend_out=False,      # Make legend inside the plot
                   palette=sns.color_palette("hls", n_colors=2)
                  )

sns.catplot(kind='box',          # Boxplot
                   x='Gender',          # X-axis - first factor
                   y='LoanAmount',      # Y-axis - values for boxplot
                   #hue='Education',    # Second factor denoted by color
                   data=data.loc[data.LoanAmount.notnull()],           # Dataframe 
                   height=8,              # Figure size (x100px)      
                   aspect=1.5,          # Width = size * aspect 
                   legend_out=False,    # Make legend inside the plot
                   palette=sns.color_palette("hls", n_colors=2)
                  )

plt.subplots_adjust(wspace=.4, hspace=.3)

## Few Observations from the Plots

-  We can see that there is no substantial difference between the median income of graduates and non-graduates.

-  Similarly, there is no significant difference between the median income of males and females.

-  However, there are a higher number of graduates (respectively, males) with very high incomes, which are appearing to be the outliers.

-  The presence of outliers in all these three numerical variables (<code>**ApplicantIncome**</code>, <code>**CoapplicantIncome**</code>, and <code>**LoanAmount**</code>), as well as missing values (only for <code>**LoanAmount**</code>) require some data munging steps (more on this later). 

In [ ]:
# Let's now plot the pairwise relationship between our continuous-valued features
sns.pairplot(data.loc[data.LoanAmount.notnull(),
                          ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']],
                 kind="reg",
                 diag_kind='kde', 
                 diag_kws={'shade': True, 'color': '#ff6600'}, 
                 plot_kws={'color': '#ff6600'})

## How Continuous-valued Features Relate to Each Other

-  As <code>**LoanAmount**</code> increases, so do <code>**ApplicantIncome**</code> (or <code>**CoapplicantIncome**</code>) increases.

-  Apparently, though, there is no strong (linear) relationship between <code>**ApplicantIncome**</code> and <code>**CoapplicantIncome**</code>. 

## 2.1 Analysis of Data Distributions: Categorical Values

-  Let's visualize the distributions of the **8 categorical** features (except the target class label):
    - <code>**Gender**</code>
    - <code>**Married**</code>
    - <code>**Dependents**</code>
    - <code>**Education**</code>
    - <code>**Self_Employed**</code>
    - <code>**Loan_Amount_Term**</code>
    - <code>**Credit_History**</code>
    - <code>**Property_Area**</code>

In [ ]:
# Let's see the frequency counts of the first three categorical variable
# Try to do the same for the remaining 5
# 'Gender'
print(data.Gender.value_counts())
print()

# 'Married'
print(data.Married.value_counts())
print()

# 'Dependents'
print(data.Dependents.value_counts())

In [ ]:
# For categorical variables, 'countplot' is the way to go
# Create a Figure containing 2x4 subplots
fig, axes = plt.subplots(2, 4, figsize=(20,10))

# Plots
sns.countplot(data.loc[data.Gender.notnull()]['Gender'], ax=axes[0,0])
sns.countplot(data.loc[data.Married.notnull()]['Married'], ax=axes[0,1])
sns.countplot(data.loc[data.Dependents.notnull()]['Dependents'], ax=axes[0,2])
sns.countplot(data.loc[data.Education.notnull()]['Education'], ax=axes[0,3])
sns.countplot(data.loc[data.Self_Employed.notnull()]['Self_Employed'], ax=axes[1,0])
sns.countplot(data.loc[data.Loan_Amount_Term.notnull()]['Loan_Amount_Term'].map(lambda x: int(x/12)), ax=axes[1,1])
sns.countplot(data.loc[data.Credit_History.notnull()]['Credit_History'], ax=axes[1,2])
sns.countplot(data.loc[data.Property_Area.notnull()]['Property_Area'], ax=axes[1,3])

plt.subplots_adjust(wspace=.5, hspace=.4)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(20,10))
cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
        'Loan_Amount_Term', 'Credit_History', 'Property_Area']
i = 0
for c in cols:
    tmp_data = pd.crosstab(data.loc[:, c], data.Loan_Status)
    # pandas.crosstab returns an mxn table where m is the number of values for the first argument (x) 
    # and n for the second argument (y)
    # As the second argument is always data.Loan_Status, n = 2 (Loan_Status is binary!)
    # e.g., x = 'Credit_History'; y = 'Loan_Status'
    # the following apply is used to transform the crosstab into a "normalized" table as follows:
    # each entry in the table displays how the i-th categorical value of x (i.e., i-th row) is distributed across
    # all the possible values of y (i.e., Y/N)
    tmp_data = tmp_data.apply(lambda x: x/tmp_data.sum(axis=1))
    tmp_data.plot.bar(stacked=True, color=['red','green'], grid=False, ax=axes[i//4, i % 4], legend=True)
    i += 1

plt.subplots_adjust(wspace=.3, hspace=.4)

## A few observations from the plots

-  Crosstabs are useful to see how "discriminant" each (categorical) feature is.

-  In a way, they provide us with some rudimental classifier.

-  Take a look at <code>**Credit_History**</code> and how this relates to <code>**Loan_Status**</code>: this feature seems quite discriminant in predicting whether or not an applicant will be given a loan approval (applicants with no credit history have their loan applications denied most of the time, i.e., ~92% of the time)

# 3. Data Preprocessing (Munging)

## Summary of the Issues

-  From our exploratory data analysis above, **two** main issues are observed:
    1. The presence of **missing values** both on numerical and categorical variables.
    2. The presence of **extreme values** on numerical variables.
-  In addition to those, we might also need to consider how to properly handle different feature's scale as well as the fact that we are in presence of both continuous and categorical attributes.

## 3.1 Handling Missing Values (NA)

-  We distinguish between **numerical** and **categorical** features.

-  One possible strategy to impute missing values is the following: replace any missing value on a numerical feature with its observed **mean**, and that of a categorical feature with its **mode**.

-  The above strategy works as long as numerical features do not contain extreme values (i.e., _outliers_). If that is the case (like ours) a better solution will be to replace missing values with the **median** of the observations (much more robust to the presence of outliers).

In [ ]:
# is_numeric_dtype(pandas.Series) returns True iff the dtype associated with the pandas.Series is numeric
from pandas.api.types import is_numeric_dtype

new_data = data.apply(lambda x: x.fillna(x.median()) 
                      if is_numeric_dtype(x) 
                      else x.fillna(x.mode().iloc[0]))

In [ ]:
new_data.describe(include='all')

In [ ]:
data.describe(include='all')

In [ ]:
print(data[data.isnull().any(axis=1)].head(10))
print(new_data[data.isnull().any(axis=1)].head(10))

In [ ]:
# Let's just assign new_data to data
data = new_data

## 3.2 Handling Outliers

-  There are several **outliers** on our numerical features <code>**ApplicantIncome**</code>, <code>**CoapplicantIncome**</code>, and <code>**LoanAmount**</code>.

-  Like missing values, outliers can be simply discarded as well (i.e., a process which is also known as **trimming** or **truncation**).

-  Another approach is called **winsorizing** and consists of replacing outliers with a specified percentile of the data (e.g., a 90% winsorization would see all data below the 5th percentile set to the 5th percentile, and data above the 95th percentile set to the 95th percentile).

In [ ]:
# Let's winsorize 'ApplicantIncome', 'CoapplicantIncome', and 'LoanAmount'
stats.mstats.winsorize(data.ApplicantIncome, limits=0.05, inplace=True)
stats.mstats.winsorize(data.CoapplicantIncome, limits=0.05, inplace=True)
stats.mstats.winsorize(data.LoanAmount, limits=0.05, inplace=True)

In [ ]:
data.describe(include='all')

In [ ]:
# Create a Figure containing 1x3 subplots
fig, axes = plt.subplots(1, 3, figsize=(14,8))

y_labeler(axes)

sns.boxplot(data.ApplicantIncome, color='#808080', ax=axes[0], orient="v")
sns.boxplot(data.CoapplicantIncome, color='#0033cc', ax=axes[1], orient="v")
sns.boxplot(data.loc[data.LoanAmount.notnull(), 'LoanAmount'], 
                color='#df2020', ax=axes[2], orient="v")
plt.subplots_adjust(wspace=.4, hspace=.3)

## A few observations from the plot

-  It seems we have reduced significantly the number of outliers but still there are some...

-  In this case, another possible solution is to nullify (or, at least, reduce) the effect of outliers by applying a log-transformation.

-  Intuitively, applying a log-transformation to a set of observations containing extreme values has the effect of "shrinking" the whole distribution (of course, independently of the basis of the logarithm).

-  To convince yourself of why this is the case, just think what happens to the values $\{1, 5, 10, 20, 1000\}$ when you transform those by applying a $\log_{10}$ operator: $\{\log_{10}(1), \log_{10}(5), \log_{10}(10), \log_{10}(20), \log_{10}(1000)\} = \{0, 0.69, 1, 1.30, 3\}$

In [ ]:
# Apply log-transformation to 'ApplicantIncome' and assign it to a new column
data['Log_ApplicantIncome'] = data.ApplicantIncome.apply(np.log)

# Apply log-transformation to 'LoanAmount' and assign it to a new column
data['Log_LoanAmount'] = data.LoanAmount.apply(np.log)
data

In [ ]:
# Create a Figure containing 1x2 subplots
fig, axes = plt.subplots(1, 2, figsize=(12,6))

# Call the vectorized function for labeling all the y-axes
y_labeler(axes)

# Plot 'ApplicantIncome' on the first subplot
sns.distplot(data.Log_ApplicantIncome, color='#808080', ax=axes[0], 
                 hist_kws=dict(edgecolor="#404040", linewidth=1))
# Plot 'LoanAmount' (limited only to non-NA values) on the second and last subplot
sns.distplot(data.loc[data.Log_LoanAmount.notnull(), 'Log_LoanAmount'], 
                 color='#df2020', ax=axes[1], 
                 hist_kws=dict(edgecolor="#404040", linewidth=1))

# Adjust space between plots
plt.subplots_adjust(wspace=.3, hspace=.3)

## 3.3 Encoding Categorical Features

-  You should remember from our last lecture that categorical feature values might need to be transformed into numerical ones before they can be fed into the ML pipeline

-  In fact, some ML algorithms can support categorical values without further manipulation but there are many others that do not. 

-  We already mentioned two main approaches to to turn categorical into numerical values: Label Encoding and One-Hot Encoding

-  The former simply assigns a number to each category, whereas the latter transform each $k$-valued categorical feature into a $k$-dimensional binary vector.

In [ ]:
# In pandas we can achieve easily one-hot encoding using the 'get_dummies()' function
categorical_features = [col for col in data.columns if not is_numeric_dtype(data[col]) and col != 'Loan_Status']
print(categorical_features)
data_with_dummies = pd.get_dummies(data, columns = categorical_features)
data_with_dummies.head()

In [ ]:
# Just as a convention, I prefer to place the column to be predicted
# as the last one.
columns = data_with_dummies.columns.tolist()
# Popping out 'mpg' from the list and insert it back at the end.
columns.insert(len(columns), columns.pop(columns.index('Loan_Status')))
# Let's refactor the DataFrame using this new column index
data_with_dummies = data_with_dummies.loc[:, columns]
data_with_dummies.head()

## Encoding Binary Class Label

In [ ]:
data = data_with_dummies
data.Loan_Status = data.Loan_Status.map(lambda x: 1 if x=='Y' else -1)
data.head()